<a href="https://colab.research.google.com/github/Hamoda-dabbit/Mining---classification-in-Arabic-Article/blob/main/CNN/CNN_TFidf_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 1- إستيراد المكتبات

In [67]:
import pandas as pd
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
import nltk
# fix random seed for reproducibility
seed = 42
np.random.seed(seed)

import string
import re
from nltk.stem.isri import ISRIStemmer
import warnings
warnings.filterwarnings("ignore")

--------

2- استيراد ملفات البيانات

In [68]:
dataset = pd.read_excel("drive/MyDrive/mining/1.xlsx")
# dataset = pd.read_excel("drive/MyDrive/mining/2.xlsx")
# dataset = pd.read_excel("drive/MyDrive/mining/3.xlsx")
# datasetf = pd.read_excel("drive/MyDrive/mining/4.xlsx")

dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17500 entries, 0 to 17499
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    17361 non-null  object
 1   targe   17500 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 273.6+ KB


------
3- shuffle  
بعثرة البيانات

In [69]:
from sklearn.utils import shuffle
dataset=shuffle(dataset)
dataset

,text,targe
1411,علمت الصباح أن الفيلم الجديد للمخرج نبيل عيوش ...,0
7555,يستعد المغرب لإطلاق أول دفعة من السندات الإسلا...,2
4930,استقلال القاضي هدف ثابت وحتمي لتحقيق العدالة ف...,1
10054,بلغ حجم الإعفاءات الضريبية خلال العام الحالي م...,2
2714,أخبارنا المغربية سناء الوردي استغلت الفنانة ال...,0
...,...,...
11284,اختتمت شبيبة حزب التجمع الوطني للأحرار أشغال ج...,3
11964,عب رت الفعاليات الثقافية والفكرية الملتئمة ضمن...,3
5390,اعترضوه بالمدينة العتيقة ومارسوا عليه الجنس بع...,1
860,تستعد لتقديم برنامجين تلفزيونيين جديدين بعد رم...,0


------
4- clean
تنظيف البيانات

In [70]:
print(np.sum(dataset.isnull().any(axis=1)))
dataset= dataset.dropna()
print(np.sum(dataset.isnull().any(axis=1)))

#-------------
def remove_hashtag(df, col = 'text'):
    for letter in r'#.][!XR':
      df[col] = df[col].astype(str).str.replace(letter,'', regex=True)
           
remove_hashtag(dataset)
#-------------
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
#-------------
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text
#-------------    
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)
#-------------
def processDocument(doc, stemmer): 

    #Replace @username with empty string
    doc = re.sub(r'@[^\s]+', ' ', doc)
    doc = re.sub(r'_', ' ', doc)
    doc = re.sub(r'\n', ' ', doc)
    doc = re.sub(r'[a-z,A-Z]', '', doc)
    doc = re.sub(r'\d', '', doc)
    #Convert www.* or https?://* to " "
    doc = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',doc)
    #Replace #word with word
    doc = re.sub(r'#([^\s]+)', r'\1', doc)
    # remove punctuations
    doc= remove_punctuations(doc)
    # normalize the tweet
    doc= normalize_arabic(doc)
    # remove repeated letters
    doc=remove_repeating_char(doc)
    #stemming
    doc = stemmer.stem(doc)
    
    return doc
    
stemmer = ISRIStemmer()
dataset["text"] = dataset['text'].apply(lambda x: processDocument(x, stemmer))
print(dataset)

reviews = dataset['text'].values
t = Tokenizer(oov_token='<UNK>')
# fit the tokenizer on the documents
t.fit_on_texts(reviews)
t.word_index['<PAD>'] = 0

VOCAB_SIZE = len(t.word_index)

139
0
                                                    text  targe
1411   علمت الصباح ان الفيلم الجديد لمخرج نبيل عيوش س...      0
7555   يستعد المغرب لاطلاق اول دفعه من السندات الاسلا...      2
4930   استقلال القاضي هدف ثابت وحتمي لتحقيق العداله ف...      1
10054  بلغ حجم الاعفاءات الضريبيه خلال العام الحالي م...      2
2714   اخبارنا المغربيه سناء الوردي استغلت الفنانه ال...      0
...                                                  ...    ...
11284  اختمت شبيبه حزب التجمع الوطني لاحرار اشغال جام...      3
11964  عب رت الفعاليات الثقافيه والفكريه الملتئمه ضمن...      3
5390   اعترضوه بالمدينه العتيقه ومارسوا عليه الجنس بع...      1
860    تستعد لتقديم برنامجين تلفزيونين جديدين بعد رمض...      0
15795  اغلق امس الثلاثاء سوق الانتقالات الصيفيه بصفقا...      4

[17361 rows x 2 columns]


In [71]:
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer


# y = dataset['targe']
# X = dataset['text']
stopwords_list = stopwords.words('arabic')

tfidf = TfidfVectorizer(max_features=500, min_df=5, max_df=0.7,token_pattern=r"(?u)\b\w\w+\b",stop_words=stopwords_list)

y1 = dataset['targe']
X1 = dataset['text']
tfidf = tfidf.fit(X1)
# X1= tfidf.fit_transform (X1) .toarray ()
TF_X = tfidf.transform(X1).toarray()
TF_X

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.08585352, 0.05430437,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.26063555, 0.        , ..., 0.        , 0.07297497,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [72]:
# print(TF_X)
xx=[]
for i in range(0,len(TF_X)):
  xx1=[]
  for j in TF_X[i]:
    if j>0:
      xx1.append(j)
  xx.append(xx1)

In [73]:
max_len=0
for i in xx:
  if max_len<len(i):
    max_len=len(i)
max_len

187

In [49]:
# for i in range(0,len(xx)):
#   for j in range(len(xx[i]),max_len):
#     xx[i].append(0)

In [50]:
# arrx = np.array(xx)
# arrx.shape

(17361, 354)

In [78]:
# VOCAB_SIZE = len(t.word_index)
VOCAB_SIZE =len(tfidf.vocabulary_)
MAX_SEQUENCE_LENGTH = max_len#1500
EMBED_SIZE = max_len#300
EPOCHS=25
BATCH_SIZE=128
# create the model
model = Sequential()
model.add(Embedding(VOCAB_SIZE, EMBED_SIZE, input_length=MAX_SEQUENCE_LENGTH))
model.add(Conv1D(filters=128, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))

# model.add(Dense(1, activation='sigmoid'))
model.add(Dense(5, activation='sigmoid'))
# model.add(Flatten())
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 187, 187)          93500     
                                                                 
 conv1d_12 (Conv1D)          (None, 187, 128)          95872     
                                                                 
 max_pooling1d_12 (MaxPoolin  (None, 93, 128)          0         
 g1D)                                                            
                                                                 
 conv1d_13 (Conv1D)          (None, 93, 64)            32832     
                                                                 
 max_pooling1d_13 (MaxPoolin  (None, 46, 64)           0         
 g1D)                                                            
                                                                 
 conv1d_14 (Conv1D)          (None, 46, 32)           

In [75]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def calculate_results(y_true, y_pred):
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [76]:
def emp_zero(X1):
  TF_X = tfidf.transform(X1).toarray()
  xx=[]
  for i in range(0,len(TF_X)):
    xx1=[]
    for j in TF_X[i]:
      if j>0:
        xx1.append(j)
    xx.append(xx1)

  for i in range(0,len(xx)):
    for j in range(len(xx[i]),max_len):
      xx[i].append(0)
    
  arx = np.array(xx)
  return arx

In [ ]:
# train_x = X1[:14000]
# train_y = y1[:14000]
# test_x = X1[14000:]
# test_y = y1[14000:]

In [ ]:
%%time
from sklearn.model_selection import StratifiedKFold
from keras.utils.np_utils import to_categorical
import os.path
from tensorflow import keras
# from sklearn import tree
y1 = dataset['targe']
X1 = dataset['text'].values
strtfdKFold = StratifiedKFold(n_splits=3)#5
kfold = strtfdKFold.split(X1, y1)
# clf = tree.DecisionTreeClassifier(random_state=5)
result_CNNClassifier=[]
#
for k, (train_index, test_index) in enumerate(kfold):
    # if(os.path.exists("drive/MyDrive/mining/CNN.net")):
    #   print("load net")
    #   model = keras.models.load_model('drive/MyDrive/mining/CNN.net') 

    X_train, X_test = X1[train_index], X1[test_index]
    X_train=emp_zero(X_train)
    X_test=emp_zero(X_test)

    y_train, y_test = y1.iloc[train_index], y1.iloc[test_index]
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    # Fit the model
    model.fit(X_train, y_train, validation_split=0.1, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1)
    y_pred=(model.predict(X_test))
    y_classes_pre = [np.argmax(y, axis=None, out=None) for y in y_pred]
    y_classes_test = [np.argmax(y, axis=None, out=None) for y in y_test]
    result_CNNClassifier.append(calculate_results(y_classes_test, y_classes_pre))
    # model.save('drive/MyDrive/mining/CNN.net')
    # print("save net")

result_CNNClassifier

Epoch 1/25
82/82 [==============================] - 36s 425ms/step - loss: 0.5118 - accuracy: 0.1960 - val_loss: 0.5006 - val_accuracy: 0.2021
Epoch 2/25
14/82 [====>.........................] - ETA: 28s - loss: 0.5013 - accuracy: 0.2042

In [1]:
accuracy=0
f1=0
precision=0
recall=0
for i in result_CNNClassifier:
  accuracy+=(i['accuracy'])
  f1+=(i['f1'])
  precision+=(i['precision'])
  recall+=(i['recall'])

accuracy=accuracy/len(result_CNNClassifier)
f1=f1/len(result_CNNClassifier)
precision=precision/len(result_CNNClassifier)
recall=recall/len(result_CNNClassifier)
# (result_DecisionTreeClassifier)
results_CNNClassifier=[{'accuracy':accuracy,'f1':f1,'precision':precision,'recall':recall}]
results_CNNClassifier

NameError: ignored

In [ ]:
print('DecisionTreeClassifier results in dataset 1:')
results_DecisionTreeClassifier

In [ ]:
##################################

In [ ]:
# # build train and test datasets
# reviews = dataset['text'].values
# sentiments = dataset['targe'].values

# # from sklearn.model_selection import train_test_split
# # test_reviews, test_sentiments, train_reviews, train_sentiments = train_test_split(reviews, sentiments, test_size=0.2)

# train_reviews = reviews[:14000]
# train_sentiments = sentiments[:14000]
# test_reviews = reviews[14000:]
# test_sentiments = sentiments[14000:]

# # y_cat = y[:14000]
# # train_reviews = reviews[:35000]
# # test_reviews = reviews[35000:]
# # y_test = y[14000:]

# norm_train_reviews = (train_reviews)
# norm_test_reviews = (test_reviews)


# le = LabelEncoder()
# y_train = le.fit_transform(train_sentiments)
# y_test = le.transform(test_sentiments)

# # print(y_train)
# # y_test = to_categorical(y_test)
# # y_cat

# #convert text to numper 
# num_classes=5 #  -> 0,  -> 1 , 2, 3, ,4

# # print(y[0])
# #convert numper to array  1=>[0,1,0,0,0]
# #Converting prediction to categorical
# from keras.utils.np_utils import to_categorical
# y_train = to_categorical(y_train)
# y_test = to_categorical(y_test)
# # print(y[0])

In [ ]:
# print(len(test_reviews))
# print(len(test_sentiments))
# print(len(train_reviews))
# print(len(train_sentiments))

## Preprocessing

To prepare text data for our deep learning model, we transform each review into a sequence.
Every word in the review is mapped to an integer index and thus the sentence turns into a sequence of numbers.

To perform this transformation, keras provides the ```Tokenizer```

In [ ]:
# t = Tokenizer(oov_token='<UNK>')
# # fit the tokenizer on the documents
# t.fit_on_texts(norm_train_reviews)
# t.word_index['<PAD>'] = 0
# # max([(k, v) for k, v in t.word_index.items()], key = lambda x:x[1]), min([(k, v) for k, v in t.word_index.items()], key = lambda x:x[1]), t.word_index['<UNK>']
# train_sequences = t.texts_to_sequences(norm_train_reviews)
# test_sequences = t.texts_to_sequences(norm_test_reviews)
# print("Vocabulary size={}".format(len(t.word_index)))
# print("Number of Documents={}".format(t.document_count))


In [ ]:
# len(t.word_index)

### Sequence Normalization

Not all reviews are of same length. To handle this difference in length of reviews, we define a maximum length.
For reviews which are smaller than this length, we pad them with zeros which longer ones are truncated

In [ ]:
# MAX_SEQUENCE_LENGTH = 1000
# # pad dataset to a maximum review length in words
# X_train = sequence.pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
# X_test = sequence.pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
# X_train.shape, X_test.shape

In [ ]:
# X_train, y_train

### Encoding Labels

The dataset contains labels of the form positive/negative. The following step encodes the labels using ```sklearn's``` ```LabelEncoder```

## Prepare the Model

Since textual data is a sequence of words, we utilize ```1D``` convolutions to scan through the sentences.
The model first transforms each word into lower dimensional embedding/vector space followed by 1d convolutions and then passing the data through dense layers before the final layer for classification

In [ ]:
# VOCAB_SIZE = len(t.word_index)
# EMBED_SIZE = 300
# EPOCHS=2
# BATCH_SIZE=128
# # create the model
# model = Sequential()
# model.add(Embedding(VOCAB_SIZE, EMBED_SIZE, input_length=MAX_SEQUENCE_LENGTH))
# model.add(Conv1D(filters=128, kernel_size=4, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Conv1D(filters=64, kernel_size=4, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Flatten())
# model.add(Dense(256, activation='relu'))

# # model.add(Dense(1, activation='sigmoid'))
# model.add(Dense(5, activation='sigmoid'))
# # model.add(Flatten())
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.summary()

## Model Training

In [ ]:
# # Fit the model
# model.fit(X_train, y_train, 
#           validation_split=0.1,
#           epochs=EPOCHS, 
#           batch_size=BATCH_SIZE, 
#           verbose=1)

## Model Evaluation

In [ ]:
# # Final evaluation of the model
# scores = model.evaluate(X_test, y_test, verbose=1)
# print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
# from sklearn.metrics import accuracy_score, precision_recall_fscore_support
# def calculate_results(y_true, y_pred):
#   # Calculate model accuracy
#   model_accuracy = accuracy_score(y_true, y_pred) * 100
#   # Calculate model precision, recall and f1 score using "weighted" average
#   model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
#   model_results = {"accuracy": model_accuracy,
#                   "precision": model_precision,
#                   "recall": model_recall,
#                   "f1": model_f1}
#   return model_results

In [ ]:
# y_pred=(model.predict(X_test))
# y_classes_pre = [np.argmax(y, axis=None, out=None) for y in y_pred]
# y_classes_test = [np.argmax(y, axis=None, out=None) for y in y_test]

# result_CNN=(calculate_results(y_classes_test, y_classes_pre))
# result_CNN

{'accuracy': 95.8048199940494,
 'f1': 0.9581779967289312,
 'precision': 0.9585923509457559,
 'recall': 0.958048199940494}

---------------


In [ ]:
# predictions = model.predict_classes(X_test).ravel()
# predictions[:10]

In [ ]:
# predictions = ['positive' if item == 1 else 'negative' for item in predictions]
# predictions[:10]

In [ ]:
# from sklearn.metrics import confusion_matrix, classification_report

# labels = ['negative', 'positive']
# print(classification_report(test_sentiments, predictions))
# pd.DataFrame(confusion_matrix(test_sentiments, predictions), index=labels, columns=labels)